# Lesson 07 Lab Exercises

In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from openpyxl import load_workbook
# DIR = '/Users/fernandojuarez/Documents/GitHub/Quantitative_Biological_Research_Python_course/'
DIR = '/Users/fernandojuarez/Documents/GitHub/Quantitative_Biological_Research_Python_course/QBRP/'

# Exercise 1

Load viperdb.csv with Pandas.

Call simple Pandas functions to achieve the following tasks:

In [177]:
df = pd.read_csv(os.path.join(DIR, '07_Lesson/Files/viperdb.csv'))

## Part A
Find the virus record with the largest outer capsid radius.

In [178]:
# print('The virus with the largest outer capsid radius of %d is %s' %(df['Outer Radius'].max(), df['Title'][df['Outer Radius'].idxmax()]))
print(f"The virus with the largest outer capsid radius of {df['Outer Radius'].max()} is {df['Title'][df['Outer Radius'].idxmax()]}.")

The virus with the largest outer capsid radius of 929 is PBCV-1 virus capsid, quasi-atomic model.


## Part B
Calculate the correlation between inner and outer capsid radius.

In [11]:
print(f"The correlation between the inner and outer capsid radius is {df['Inner Radius'].corr(df['Outer Radius'])}")
print('The correlation between the inner and outer capsid radius is %0.3f' %(df['Inner Radius'].corr(df['Outer Radius'])))

The correlation between the inner and outer capsid radius is 0.9353034053307225
The correlation between the inner and outer capsid radius is 0.935


## Part C
Which viruses are larger? dsDNA or ssRNA?

In [ ]:
contains_dsDNA = df['Genome'].str.contains('dsDNA', na=False)
contains_ssRNA = df['Genome'].str.contains('ssRNA', na=False)

filter_for_dsDNA = df[contains_dsDNA]
filter_for_ssRNA = df[contains_ssRNA]

print('The average outer capsid radius for dsDNA viruses is %0.2f units' %(filter_for_dsDNA['Outer Radius'].mean()))
print('The average outer capsid radius for ssRNA viruses is %0.2f units' %(filter_for_ssRNA['Outer Radius'].mean()))

The average outer capsid radius for dsDNA viruses is 293.46 units
The average outer capsid radius for ssRNA viruses is 174.35 units


## Part D
Assuming that viral capsids are roughly spherical, their surface area can be calculated by the formula 𝐴 =
4𝜋𝑅^2 where 𝑅 is the outer radius. Calculate this term for all virus records and add it to the DataFrame as
a new column. How is it correlated with the Outside SASA field?

In [43]:
# Apply a function to a column and add it as a new column
# Find correlation between outside SASA field and new column

df['Surface Area'] = df['Outer Radius'].map(lambda x: 4*np.pi*(x**2))

print('The correlation between the estimated viral capsid Surface Area and the Outside SASA'\
      ' is %0.3f units.' %(df['Surface Area'].corr(df['Outside SASA'])))

The correlation between the estimated viral capsid Surface Area and the Outside SASA is 0.424 units.


# Exercise 2

## Part A
Download the PDF file cancer_facts.pdf and extract the table “Estimated Number of New Cases for
Selected Cancers by State, US, 2014” into a CSV file. Use an online PDF-to-Excel tool such as:
https://www.coolutils.com/online/PDF-to-CSV.

'07_Lesson/Files/cancer_facts.xlsx'

## Part B
Which state has the highest proportion of new Leukemia cases as a fraction of all cancer types?

In [ ]:

def parse_table(header_start, colrange, row_start, row_end):
    return pd.read_excel(os.path.join(DIR, '07_Lesson/Files/cancer_facts.xlsx'),\
                          skiprows=header_start, thousands= ',', usecols=colrange, na_filter= False, nrows=(row_end-row_start))

# header_start needs to be +3 from row itentified as header or +4 from the row just before the desired header


In [4]:

df = parse_table(445,'B,D:N', 445, 497)

# Rename columns.
df.columns = ['State','All Sites', 'Female Breast', 'Uterine Cervix', 'Colon & Rectum', 'Uterine Corpus', 'Leukemia',\
               'Lung & Bronchus', 'Melanoma of the Skin', 'Non-Hodgkins Lymphoma', 'Prostate', 'Urinary Bladder']

## This was all done while fiddling with dtype assignment. Ultimately all fixed with thousands arg set to ','
# s_numeric = pd.to_numeric(df['Leukemia'],errors='coerce')
# display(df['Female Breast'].dtype)
# display(df['Leukemia'])
# print(f"\nConverted dtype: {s_numeric.dtype}")
# display(s_numeric)

# divide leukemia col by all sites col then _.idmax() and .max() it to display the state
print('%s has the highest proportionn of new Leukemia cases (%0.2f) to total cancer types.'\
       %(df['State'][(df['Leukemia']/df['All Sites']).idxmax()], (df['Leukemia']/df['All Sites']).max())) 

Colorado has the highest proportionn of new Leukemia cases (0.04) to total cancer types.


## Part C
What is the percentage of new cancer cases not classified as one of the provided types per state?

In [6]:
# add all provided types in US and subtract from total?

# display(df.tail(1))
# print(pd.to_numeric(df.iloc[-1, 2:]).sum())
# print(df['All Sites'][51])
# print(df['All Sites'][51]-pd.to_numeric(df.iloc[-1, 2:]).sum())

# print('%0.2f %/ of the new cancer cases in the United States are not classified
      
# OH THIS IS COMPLETELY DIFFERENT
# df.iloc[:50, 2:].sum(axis=0)
# df['Outer Radius'].map(lambda x: 4*np.pi*(x**2))

# Replacing symbol values with upper value of range represented by symbol
df['Uterine Cervix'] = pd.to_numeric(df['Uterine Cervix'].map(lambda x: 50 if x=='†' else x))

# Percentage of new cases not classified by provided types per state
df['Unclassified [%]'] = ((df['All Sites'] - df.iloc[:, 2:].sum(axis=1))/df['All Sites']*100).round(2)

display(df[['State', 'Unclassified [%]']])

,State,Unclassified [%]
0,Alabama,29.85
1,Alaska,38.13
2,Arizona,32.81
3,Arkansas,31.84
4,California,31.22
5,Colorado,28.60
6,Connecticut,30.40
7,Delaware,26.32
8,Dist. of Columbia,30.28
9,Florida,26.14


## Part D
Find the two states that are most similar to each other with respect to the profile of new cancer types.

Compare the fraction of cancer types, not the absolute values.

In [62]:
# I could make a new df that is a matrix of ratios with states being both the index and the columns
# That's done by the .corr() function, just need to format a new df to apply it to

# Calculate fraction of cancer types per state
# I want it to go do this all throughout once column then move on to the next, 
# divide values in col by value in all sites then move on to the next one. we did that with iloc right?

def all_sites_ratio(df_column):
    return df_column/df['All Sites']

df_ratios = df.iloc[:,1:-1].apply(all_sites_ratio)

# rename index to states
# transpose subdf of just states and cancer types so columns are now states

In [106]:
# Rename index to states
# pd.DataFrame(df_ratios, index= df['State'])
# corr_pairs = df_ratios.set_index(df['State'])
corr_pairs = df_ratios.set_index(df['State']).iloc[:-1,:].transpose().corr()

# sorted_pairs = corr_pairs[corr_pairs != 1.0]
# sorted_pairs = corr_pairs[corr_pairs != 1.0].max()
# sorted_pairs = corr_pairs[corr_pairs != 1.0].idxmax()
# sorted_pairs = corr_pairs[corr_pairs != 1.0].max().idxmax()

sorted_pairs = corr_pairs[corr_pairs != 1.0].max().sort_values(ascending=False)

print('The two states with the most similar new cancer types profiles are:\n', sorted_pairs[0:2])

The two states with the most similar new cancer types profiles are:
 State
Massachusetts    0.99995
Connecticut      0.99995
dtype: float64
